## Spectral Collocation Method

In [1]:
import numpy as np
import scipy.linalg as la
from scipy.interpolate import interp1d
import time

# Option parameters
K = 100.0       # Strike price
T = 1.0         # Time to maturity (in years)
r = 0.05        # Risk-free interest rate
q = 0.05        # Dividend yield
sigma = 0.25    # Volatility

# Computational domain
S_min = 0.0
S_max = 3 * K   # Upper bound of asset price

# Parameters from Table 2
l = 25  # Number of collocation points in asset price (S)
m = 4   # Not used directly in this implementation
n = 9   # Number of time steps

# Computational parameters
N = l           # Number of spatial collocation points
M = n * 10      # Number of time steps (adjusted for stability)

# Time step size
dt = T / M

# Generate Chebyshev-Gauss-Lobatto points in [-1, 1]
i = np.arange(N + 1)
x = np.cos(np.pi * i / N)

# Map x from [-1, 1] to [S_min, S_max]
S = 0.5 * (S_max - S_min) * (x + 1) + S_min

def chebyshev_diff_matrix(N, S):
    c = np.ones(N + 1)
    c[0] = 2.0
    c[-1] = 2.0
    c = c * (-1) ** np.arange(N + 1)
    X = np.tile(S, (N + 1, 1)).T
    dX = X - X.T + np.eye(N + 1)  # Avoid division by zero
    D = (c[:, None] / c[None, :]) / dX
    D = D - np.diag(np.sum(D, axis=1))
    return D

# Compute the first and second derivative matrices
D1 = chebyshev_diff_matrix(N, S)
D2 = D1 @ D1

# Payoff function for a put option
def payoff(S):
    return np.maximum(K - S, 0.0)

# Initial option values at maturity (t = T)
V = payoff(S)

# Precompute constants
sigma2 = sigma ** 2
q_mat = (r - q) * np.diag(S)
sigma2_mat = 0.5 * sigma2 * np.diag(S ** 2)

# Construct the operator matrix L
L = -sigma2_mat @ D2 - q_mat @ D1 + r * np.eye(N + 1)

# Time-stepping parameters
omega = 1.2  # Over-relaxation parameter for PSOR
tolerance = 1e-8
max_iterations = 10000

# Record the start time
start_time = time.time()

# Time-stepping loop
for n in range(M):
    # Current time
    t = T - n * dt

    # Right-hand side
    b = V.copy()

    # Implicit time-stepping matrix
    A = np.eye(N + 1) + dt * L

    # Apply boundary conditions
    A[0, :] = 0.0
    A[0, 0] = 1.0
    b[0] = K * np.exp(-r * (T - t))

    A[-1, :] = 0.0
    A[-1, -1] = 1.0
    b[-1] = 0.0

    # Initialize V_new for PSOR
    V_new = V.copy()

    # PSOR method to solve A V_new = b with inequality constraints
    for iteration in range(max_iterations):
        V_old = V_new.copy()
        for i in range(1, N):  # Exclude boundary points
            residual = (A[i, :] @ V_new) - b[i]
            V_new[i] = V_new[i] - omega * residual / A[i, i]
            # Apply early exercise constraint
            V_new[i] = max(V_new[i], payoff(S[i]))
        # Check convergence
        if np.linalg.norm(V_new - V_old, np.inf) < tolerance:
            break
    else:
        print(f"Warning: PSOR did not converge at time step {n}")

    # Update the option values
    V = V_new.copy()

# Interpolate to find V at S = K
V_func = interp1d(S, V, kind='cubic')
V_at_K = V_func(K)

# Compute the American premium
# For accurate premium, subtract the European option price
# For now, we'll assume the European option price is known
# European option price using Black-Scholes formula
from scipy.stats import norm

def european_put_price(S, K, T, r, q, sigma):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    return price

# Compute European option price at S = K
Euro_price = european_put_price(K, K, T, r, q, sigma)

# American premium
American_premium = V_at_K - Euro_price

# Execution time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print(f"Option Value at S = K: {V_at_K:.12f}")
print(f"European Option Price: {Euro_price:.12f}")
print(f"American Premium: {American_premium:.12f}")
print(f"Execution Time: {execution_time:.2e} seconds")


Option Value at S = K: 9.592299211140
European Option Price: 9.462492596167
American Premium: 0.129806614972
Execution Time: 5.65e-01 seconds
